In [13]:
from rl.environments import Environment

filename = 'WARF_00_00_XX_d_90_dose_9_INR_9_T_DQN_(20,20)_g_0.95_e_func_lr_0.01_buff_90_clr_F_btch_30_vld_0.3'

env = Environment(filename=filename)
agent = env._agent['protocol']
subject = env._subject['W']


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(30, 20), dtype=float32) is not an element of this graph.

In [8]:
temp = env.trajectory()['protocol']

In [9]:
print('patient: Age: {}, CYP2C9: {}, VKORC1: {}'.format(
      temp.state[0].value['Age'],
      temp.state[0].value['CYP2C9'],
      temp.state[0].value['VKORC1'])
     )
print('day\tINR\taction\treward')
for t in temp.iterrows():
    print('{}\t{:3.1f}\t{:5.1f}\t{:3.2f}'.format(t[0],
                                              t[1].state.value['INRs'][-1],
                                              t[1].action.value['value'],
                                              t[1].reward))

patient: Age: 82, CYP2C9: *1/*1, VKORC1: A/A
day	INR	action	reward
0	1.0	 15.0	0.00
1	1.6	 15.0	0.00
2	2.0	 15.0	1.00
3	2.2	 15.0	1.00
4	2.6	 15.0	1.00
5	2.8	 15.0	1.00
6	3.0	 15.0	0.00
7	3.5	 15.0	0.00
8	3.8	 15.0	0.00
9	3.9	 15.0	0.00
10	4.2	 15.0	0.00
11	4.3	 15.0	0.00
12	4.2	 15.0	0.00
13	4.7	 15.0	0.00
14	4.7	 15.0	0.00
15	4.7	 15.0	0.00
16	5.0	 15.0	0.00
17	5.2	 15.0	0.00
18	5.1	 15.0	0.00
19	5.4	 15.0	0.00
20	5.4	 15.0	0.00
21	5.3	 15.0	0.00
22	5.7	 15.0	0.00
23	5.7	 15.0	0.00
24	5.8	 15.0	0.00
25	5.9	 15.0	0.00
26	5.7	 15.0	0.00
27	6.3	 15.0	0.00
28	5.7	 15.0	0.00
29	5.9	 15.0	0.00
30	5.9	 15.0	0.00
31	5.9	 15.0	0.00
32	6.2	 15.0	0.00
33	5.9	 15.0	0.00
34	6.0	 15.0	0.00
35	6.2	 15.0	0.00
36	6.2	 15.0	0.00
37	6.1	 15.0	0.00
38	6.0	 15.0	0.00
39	5.7	 15.0	0.00
40	6.1	 15.0	0.00
41	6.0	 15.0	0.00
42	6.1	 15.0	0.00
43	6.4	 15.0	0.00
44	6.3	 15.0	0.00
45	6.3	 15.0	0.00
46	6.1	 15.0	0.00
47	6.2	 15.0	0.00
48	6.2	 15.0	0.00
49	5.9	 15.0	0.00
50	6.3	 15.0	0.00
51	6.2	 15.0	0.00
52	6.1	

In [10]:
TTRs = {}
actions = {}
for _ in range(100):
    temp = env.trajectory()['protocol']
    try:
        TTR = sum(1 if 2.0<=t[1].state.value['INRs'][-1]<=3.0 else 0 for t in temp.iterrows()) / len(temp)
        ave_action = sum(t[1].action.value['value'] for t in temp.iterrows()) / len(temp)
        TTRs[(temp.state[0].value['Age'],
                 temp.state[0].value['CYP2C9'],
                 temp.state[0].value['VKORC1'])].append(TTR)
        actions[(temp.state[0].value['Age'],
                temp.state[0].value['CYP2C9'],
                temp.state[0].value['VKORC1'])].append(ave_action)
    except KeyError:
        TTRs[(temp.state[0].value['Age'],
                 temp.state[0].value['CYP2C9'],
                 temp.state[0].value['VKORC1'])] = [TTR]
        actions[(temp.state[0].value['Age'],
                 temp.state[0].value['CYP2C9'],
                 temp.state[0].value['VKORC1'])] = [ave_action]

print('TTR\npatient\t\t\tcount\tmin\taverage\tmax')
for patient, r in TTRs.items():
    print('{}\t{:3}\t{:2.2%}\t{:2.2%}\t{:2.2%}'.format(patient, len(r), min(r), sum(r)/len(r), max(r)))

print('actions\npatient\t\t\tcount\tmin\taverage\tmax')
for patient, r in actions.items():
    print('{}\t{:3}\t{:3.2}\t{:3.2}\t{:3.2}'.format(patient, len(r), min(r), sum(r)/len(r), max(r)))

TTR
patient			count	min	average	max
(77, '*2/*3', 'A/A')	  2	1.11%	1.67%	2.22%
(79, '*3/*3', 'G/A')	  1	2.22%	2.22%	2.22%
(80, '*1/*2', 'G/G')	  1	4.44%	4.44%	4.44%
(71, '*3/*3', 'G/G')	  1	4.44%	4.44%	4.44%
(79, '*1/*1', 'A/A')	  1	2.22%	2.22%	2.22%
(84, '*1/*2', 'G/A')	  1	46.67%	46.67%	46.67%
(80, '*2/*3', 'G/G')	  2	2.22%	2.22%	2.22%
(77, '*2/*2', 'A/A')	  1	3.33%	3.33%	3.33%
(71, '*1/*1', 'G/A')	  1	14.44%	14.44%	14.44%
(77, '*1/*3', 'G/G')	  1	26.67%	26.67%	26.67%
(74, '*2/*3', 'A/A')	  1	2.22%	2.22%	2.22%
(70, '*2/*3', 'G/A')	  1	15.56%	15.56%	15.56%
(78, '*2/*3', 'G/A')	  1	8.89%	8.89%	8.89%
(80, '*3/*3', 'A/A')	  1	2.22%	2.22%	2.22%
(70, '*1/*2', 'G/A')	  2	3.33%	4.44%	5.56%
(80, '*1/*1', 'G/G')	  1	87.78%	87.78%	87.78%
(84, '*2/*2', 'A/A')	  1	1.11%	1.11%	1.11%
(70, '*2/*2', 'A/A')	  2	0.00%	1.67%	3.33%
(85, '*2/*2', 'A/A')	  1	1.11%	1.11%	1.11%
(74, '*1/*3', 'A/A')	  1	18.89%	18.89%	18.89%
(84, '*2/*3', 'G/G')	  1	2.22%	2.22%	2.22%
(72, '*1/*2', 'A/A')	  1	2.22%	2.22%	2.22%


In [17]:
import json

j_results = {}
for p, v in results.items():
    j_results[', '.join((p[2], p[1], str(p[0])))] = v

with open("10000patients.json", "w") as write_file:
    json.dump(j_results, write_file)
    

NameError: name 'results' is not defined

In [ ]:
all_records = []
for p, value in results.items():
    all_records.extend(list((p, v) for v in value))
    
for r in all_records:
    print(r[0], r[1])

In [ ]:
import csv
csv.writer()

In [ ]:
for patient in sorted(results, key=lambda s: s[2]+s[1]+str(s[0])):
    print('\t'.join((patient[2], patient[1], str(patient[0]),
                     '{: 3}'.format(len(results[patient])),
                     '{:3.2}'.format(min(results[patient])),
                     '{:3.2}'.format(sum(results[patient])/len(results[patient])),
                     '{:3.2}'.format(max(results[patient])))))